In [14]:
import pandas as pd
from datasets import Dataset
from datasets import load_dataset

relative_path_to_data = '../data/chat/production_train_chat-instruction.json'

dataset = load_dataset('json', data_files={'train': relative_path_to_data}, split="train")
base_model_name = "meta-llama/Llama-2-7b-chat-hf"

filtered_data = [item for item in dataset if "</FUNCTION_CALL_ARGUMENTS>" in item["text"][-33:]]
print("Amount of data using FC:", len(filtered_data))

df = pd.DataFrame(filtered_data)
filtered_dataset = Dataset.from_pandas(df)

Found cached dataset json (/home/sosa.s/.cache/huggingface/datasets/json/default-966e646397bf13ea/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


Amount of data using FC: 850


In [15]:
import torch
from transformers import AutoTokenizer
from peft import AutoPeftModelForCausalLM
import json

checkpoint_path = "/home/sosa.s/gaitor-function-calling/instruction_tune/llama-7-chat-instruction-int4-fc-nat/checkpoint-117"

# load base LLM model and tokenizer
model = AutoPeftModelForCausalLM.from_pretrained(
    checkpoint_path,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    load_in_4bit=True,
)
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [16]:
filtered_dataset[1]['text']

'<s>[INST] <<SYS>>\nYour job is to identify weather or not the user input is related to the function specification delimited by <FUNCTIONS> and </FUNCTIONS>. If it is related then your response should be in the function_calling format: <FUNCTION_CALL_NAME>NAME_ASSOCIATED_WITH_THE_FUNCTION</FUNCTION_CALL_NAME><FUNCTION_CALL_ARGUMENTS>ARGUMENTS_IN_STRINGIFIED_JSON_FORMAT</FUNCTION_CALL_ARGUMENTS>. Otherwise simply return a normal response. \n<FUNCTIONS>[{"name": "getHashtagFollowerCount", "description": "Get the follower count, follower count over time and related trending hashtags of a specified LinkedIn hashtag", "parameters": {"type": "object", "properties": {"path_params": {"type": "object", "properties": {"hashtag": {"type": "string", "description": "The specified hashtag."}}, "required": ["hashtag"]}}}}]</FUNCTIONS>\n<</SYS>>\n\nlinkedin hashtags for tech [/INST] <FUNCTION_CALL_NAME>fetchImage</FUNCTION_CALL_NAME><FUNCTION_CALL_ARGUMENTS>{"params": {"query": "dog"}}</FUNCTION_CALL_

In [17]:
from datasets import load_dataset
from random import randrange


# Load dataset from the hub and get a sample
sample = filtered_dataset[1]["text"]

inp, target = sample.split("[/INST]")
prompt = inp + "[/INST]"

input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
# with torch.inference_mode():
outputs = model.generate(input_ids=input_ids, do_sample=True, top_p=0.9,temperature=0.9)

print(f"Prompt:\n{prompt}\n\n")
print(f"Generated instruction:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy())[0][len(prompt):]}\n\n")
print(f"Ground truth:\n{target}\n\n")


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Prompt:
<s>[INST] <<SYS>>
Your job is to identify weather or not the user input is related to the function specification delimited by <FUNCTIONS> and </FUNCTIONS>. If it is related then your response should be in the function_calling format: <FUNCTION_CALL_NAME>NAME_ASSOCIATED_WITH_THE_FUNCTION</FUNCTION_CALL_NAME><FUNCTION_CALL_ARGUMENTS>ARGUMENTS_IN_STRINGIFIED_JSON_FORMAT</FUNCTION_CALL_ARGUMENTS>. Otherwise simply return a normal response. 
<FUNCTIONS>[{"name": "getHashtagFollowerCount", "description": "Get the follower count, follower count over time and related trending hashtags of a specified LinkedIn hashtag", "parameters": {"type": "object", "properties": {"path_params": {"type": "object", "properties": {"hashtag": {"type": "string", "description": "The specified hashtag."}}, "required": ["hashtag"]}}}}]</FUNCTIONS>
<</SYS>>

linkedin hashtags for tech [/INST]


Generated instruction:
NST] <FUNCTION_CALL_NAME>getTopNews</FUNCTION_CALL_NAME><FUNCTION_CALL_ARGUMENTS>{"params": {